# DATA ANALYSIS PROJECT

## INDIAN START-UPS FUNDING. INSIGHTS AND TRENDS FROM 2018 TO 2021

### Project Description
This data analysis project focuses on the funding received by start-ups in India from 2018 to 2021. <br><br>The objective is to gain insights into the ecosystem and propose the best course of action for our team's venture. <br><br>By analyzing the data on funding amounts, start-up details, and investor information, we aim to unearth prevailing patterns and gain insights about the opportunities in India's start-up ecosystem to inform decision-making.

### Taking Preview of the Data at Hand

Data from four different sources were gathered for this project. Two from Microsoft SQL Server, one from onedrive and the last one from a github repo.

We set the van off with loading the needed libraries.

In [416]:
# Load some libraries

import pyodbc
from dotenv import dotenv_values
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import SimpleImputer
import re
import warnings

warnings.filterwarnings('ignore')

# Display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Convert amounts from scientific format
pd.set_option('display.float_format', '{:.1f}'.format)

In [417]:
# # Load environment variables from .env file into a dictionary

# env_var = dotenv_values('.env')

# # Get the values for the credentials you set in the '.env' file

# database = env_var.get('DATABASE')
# username = env_var.get('USERNAME')
# password = env_var.get('PASSWORD')
# server = env_var.get('SERVER')


# connection_string = f"DRIVER={{SQL Server}}; SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [418]:
# Get the values for the database credentials set in the '.env' file
env_var = {key.upper(): value for key, value in dotenv_values('.env').items()}

# Unpack the values from the dictionary
database, username, password, server = (env_var.get(key) for key in ['DATABASE', 'USERNAME', 'PASSWORD', 'SERVER'])

connection_string = f"DRIVER={{SQL Server}}; SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [419]:
# Connect to database

connection = pyodbc.connect(connection_string)

Success! Now, we read the data using sql SELECT statement and pandas' read_sql.

In [420]:
# Loading the data for 2020

query2020 = "select * from dbo.LP1_startup_funding2020"

data2020 = pd.read_sql(query2020, connection)

In [421]:
# Loading the data for 2021

query2021 = "select * from dbo.LP1_startup_funding2021"

data2021 = pd.read_sql(query2021, connection)

Perfect! We hop onto the other datasets obtained in '.csv' format

In [422]:
# Loading data for 2018 & 2019


data2018 = pd.read_csv('startup_funding2018.csv')
data2019 = pd.read_csv('startup_funding2019.csv')

### Data Inspection

### 2021 Data

In [423]:
data2021.head(10)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,"$188,000,000",None
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI","$200,000",None
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Undisclosed,Pre-series A
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,Undisclosed,None
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,"$1,000,000",None


### Issues arising from data2021:

* Founded column is a float. It has to be a date

* Some Amounts have $undisclosed, Undisclosed and undisclosed. We may treat them as missing values

* The Amounts and Stage for FanPlay company are interchanged

* At index 242, 256, 257, and 545, the amount appears at the investor’s column and then stage at the amount column

* Little Leap campany at index 538 has Ah! Ventures(investors) instead of amount and then amount at stage column. Also, ‘Holistic Development Programs for children in …; should be   replaced with Vishal Gupta as founder.

* BHyve company has part of ‘what_it_does’ at founders cl then founders at investor cl and investors at amount col ……. It also had investor’s name as ITO Angel Network instead of JITO Angel Network

* Some amounts are separated by ‘,’ some too have ‘$$’ preceding them and some only ‘$’ as amount…this is for EventBeep, MPL

* Amount for Godamwale is misspelt as 1000000\t#REF! instead of 1000000 and is at investor col with stage rather taking its place. Also, the investor is **Capt. Anand Aryamane**

* for index 1100-Sochcast company, Heaquarter is replaced with ‘Online Media\t#REF!’ and sector
* There are some duplicates

### Resolution

* Delete duplicates
* Delete rows with anomalous values
* Impute with right values from credible sources where such values are mispelt, omitted and wrongly placed
* Convert columns to right data types
* Standardise the 'Stage' column for good analysis by renaming or grouping

In [424]:
# Confirm duplicated records

data2021.duplicated().sum()

19

Let's take a look, first.

In [425]:
data2021[data2021.duplicated()].head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
107,Curefoods,2020.0,Bangalore,Food & Beverages,Healthy & nutritious foods and cold pressed ju...,Ankit Nagori,"Iron Pillar, Nordstar, Binny Bansal",$13000000,None
109,Bewakoof,2012.0,Mumbai,Apparel & Fashion,Bewakoof is a lifestyle fashion brand that mak...,Prabhkiran Singh,InvestCorp,$8000000,None
111,FanPlay,2020.0,Computer Games,Computer Games,A real money game app specializing in trivia g...,YC W21,"Pritesh Kumar, Bharat Gupta",Upsparks,$1200000
117,Advantage Club,2014.0,Mumbai,HRTech,Advantage Club is India's largest employee eng...,"Sourabh Deorah, Smiti Bhatt Deorah","Y Combinator, Broom Ventures, Kunal Shah",$1700000,None
119,Ruptok,2020.0,New Delhi,FinTech,Ruptok fintech Pvt. Ltd. is an online gold loa...,Ankur Gupta,Eclear Leasing,$1000000,None


Alright! They have to be on the go...!

In [426]:
# Drop the duplicates

data2021 = data2021.drop_duplicates()


Checking.......

In [427]:
# Check data shape before deletion
data2021.shape

(1190, 9)

Success!

### Dealing with anomalous rows

....taking a look first at the unique values of each column, <br> then take care of them


### Company_Brand column

In [428]:
print(data2021['Company_Brand'].unique().tolist())


['Unbox Robotics', 'upGrad', 'Lead School', 'Bizongo', 'FypMoney', 'Urban Company', 'Comofi Medtech', 'Qube Health', 'Vitra.ai', 'Taikee', 'Fitterfly', 'FreeStand', 'MyDentalPlan', 'Freyr Energy', 'DealShare', 'Tessolve', 'Smart Joules', 'ORAI', 'ixamBee', 'CoinSwitch Kuber', 'Razorpay', 'Uable', 'ZFW Hospitality', 'Optimized Electrotech', 'Northern Arc', 'Chara', 'Sirona Hygiene', 'Pristyn Care', 'TrulyMadly', 'Vidyakul', 'Jambox Games', 'Miko', 'SuperK', 'TagZ Foods', 'Fuel Buddy', 'M1xchange', 'Spintly', 'EF Polymer', 'boAt', 'TruNativ', 'iBus', 'Puresh Daily', 'LearnVern', 'LimeChat', 'Dybo', 'moEVing', 'WeWork India', 'Solethreads', 'Beldara', 'CredFlow', 'Do Your Thng', 'Oye Rickshaw', 'Legitquest', 'Swiggy', 'AntWak', 'PlayShifu', 'Pitstop', 'Lendingkart', 'CRED', 'Karnival', 'PharmEasy', 'WESS', 'ShopSe', 'Fantasy Akhada', 'Rage Coffee', 'Groww', 'Avanti Finance', 'Kudos', 'OfBusiness', 'Superpro.ai', 'ShareChat', 'WiT-ACE', 'Spinny', 'Alteria Capital', 'Safexpay', 'PumPumPum',

Same company name was treated as separate identities: BYJU’S and  BYJU'S. <br>This might be due to the different apostrophies used.

........let's check!

In [429]:
# Check if they are same

"BYJU'S" ==  'BYJU’S'


False

In [430]:
# Good! Now we correct that and that's all for company_Brand

data2021['Company_Brand'] = data2021['Company_Brand'].str.replace("’", "'")

In [431]:
data2021['Company_Brand'].nunique()

1032

Good! Worked! Now, let's head to Headquarter

In [432]:
print(data2021['HeadQuarter'].info())
print(data2021['HeadQuarter'].nunique())
print(data2021['HeadQuarter'].unique())

<class 'pandas.core.series.Series'>
Int64Index: 1190 entries, 0 to 1208
Series name: HeadQuarter
Non-Null Count  Dtype 
--------------  ----- 
1189 non-null   object
dtypes: object(1)
memory usage: 18.6+ KB
None
70
['Bangalore' 'Mumbai' 'Gurugram' 'New Delhi' 'Hyderabad' 'Jaipur'
 'Ahmadabad' 'Chennai' None 'Small Towns, Andhra Pradesh' 'Goa'
 'Rajsamand' 'Ranchi' 'Faridabad, Haryana' 'Gujarat' 'Pune' 'Thane'
 'Computer Games' 'Cochin' 'Noida' 'Chandigarh' 'Gurgaon' 'Vadodara'
 'Food & Beverages' 'Pharmaceuticals\t#REF!' 'Gurugram\t#REF!' 'Kolkata'
 'Ahmedabad' 'Mohali' 'Haryana' 'Indore' 'Powai' 'Ghaziabad' 'Nagpur'
 'West Bengal' 'Patna' 'Samsitpur' 'Lucknow' 'Telangana' 'Silvassa'
 'Thiruvananthapuram' 'Faridabad' 'Roorkee' 'Ambernath' 'Panchkula'
 'Surat' 'Coimbatore' 'Andheri' 'Mangalore' 'Telugana' 'Bhubaneswar'
 'Kottayam' 'Beijing' 'Panaji' 'Satara' 'Orissia' 'Jodhpur' 'New York'
 'Santra' 'Mountain View, CA' 'Trivandrum' 'Jharkhand' 'Kanpur' 'Bhilwara'
 'Guwahati' 'Online Medi

There sure have been wrong placement for 'Information Technology & Services' and  'Food & Beverages' <br>
Let's check that out!

In [433]:
# Checking .....

data2021[data2021['HeadQuarter'] == 'Food & Beverages']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
241,MasterChow,2020.0,Food & Beverages,Hauz Khas,A ready-to-cook Asian cuisine brand,"Vidur Kataria, Sidhanth Madan",WEH Ventures,$461000,Seed


In [434]:
# Placing them correctly for MasterChow

data2021.iloc[236, 2] = 'New Delhi'
data2021.iloc[236, 3] = 'Food & Beverages'

In [435]:
# Checking .....

data2021[data2021['HeadQuarter'] == 'Food & Beverages']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage


In [436]:
# Checking for Peak

data2021[data2021['HeadQuarter'] == 'Information Technology & Services']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1176,Peak,2014.0,Information Technology & Services,"Manchester, Greater Manchester",Peak helps the world's smartest companies put ...,Atul Sharma,SoftBank Vision Fund 2,$75000000,Series C


In [437]:
# Placing them correctly

data2021.iloc[1149, 2] = 'Manchester'
data2021.iloc[1149, 3] = 'Information Technology & Services'

data2021.at[1176, 'HeadQuarter'] = 'Manchester'
data2021.at[1176, 'Sector'] = 'Information Technology & Services'

In [438]:
# Checking ......

data2021[data2021['HeadQuarter'] == 'Information Technology & Services']


,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage


Alright! Let's continue dealing with some anomalous rows

Deleting rows with anomalous values


In [439]:
# Function to drop anomalous rows


def drop_rows_by_column_values(dataframe, column_name, value):
    """
    Drops rows from the specified DataFrame based on a specific column value.

    Parameters:
        dataframe (pandas.DataFrame): The DataFrame to modify.
        column_name (str): The name of the column to check for the specified value.
        value: The value to match and drop rows based on the specified column.

    Returns:
        None

    Description:
        This function removes rows from the specified DataFrame where the specified column
        matches the provided value. The rows are dropped in-place, modifying the DataFrame directly.

        Parameters:
        - dataframe (pandas.DataFrame): The DataFrame to modify.
        - column_name (str): The name of the column to check for the specified value.
        - value: The value to match and drop rows based on the specified column.

        Returns:
        - None

        Example usage:
        drop_rows_by_column_value(data2021, 'Company_Brand', 'BrandXYZ')

        This example will drop all rows from the 'data2021' DataFrame where the 'Company_Brand'
        column contains the value 'BrandXYZ'.
    """
    dataframe.drop(dataframe[dataframe[column_name].isin(value)].index, inplace=True)

In [440]:
# Invoke function to drop the list of rows

drop_rows_by_column_values(data2021, 'Company_Brand', ['Little Leap', 'Godamwale', \
        'Sochcast', 'BHyve', 'Fullife Healthcare', 'MoEVing', 'AdmitKard', 'MYRE Capital'])


In [441]:
# Making sure the rows were deleted

data2021.shape

(1182, 9)

In [442]:
# Re-arrange indices for the data frame

data2021.reset_index(drop=True, inplace=True)

In [443]:
# Check for re-indexing

data2021.tail()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1177,Gigforce,2019.0,Gurugram,Staffing & Recruiting,A gig/on-demand staffing company.,"Chirag Mittal, Anirudh Syal",Endiya Partners,$3000000,Pre-series A
1178,Vahdam,2015.0,New Delhi,Food & Beverages,VAHDAM is among the world’s first vertically i...,Bala Sarda,IIFL AMC,$20000000,Series D
1179,Leap Finance,2019.0,Bangalore,Financial Services,International education loans for high potenti...,"Arnav Kumar, Vaibhav Singh",Owl Ventures,$55000000,Series C
1180,CollegeDekho,2015.0,Gurugram,EdTech,"Collegedekho.com is Student’s Partner, Friend ...",Ruchir Arora,"Winter Capital, ETS, Man Capital",$26000000,Series B
1181,WeRize,2019.0,Bangalore,Financial Services,India’s first socially distributed full stack ...,"Vishal Chopra, Himanshu Gupta","3one4 Capital, Kalaari Capital",$8000000,Series A


### Dealing with the 'Founded' Col

In [444]:
# Imputing missing values

array = data2021['Founded'].values.reshape(-1,1)
imputer = SimpleImputer(strategy='most_frequent')

data2021['Founded'] = imputer.fit_transform(array)

In [445]:
data2021['Founded'].unique()

array([2019., 2015., 2012., 2021., 2014., 2018., 2016., 2020., 2010.,
       2017., 1993., 2008., 2013., 1999., 1989., 2011., 2009., 2002.,
       1994., 2006., 2000., 2007., 1978., 2003., 1998., 1991., 1984.,
       2004., 2005., 1963.])

Converting 'Founded' from float to int64

In [446]:
data2021['Founded'] = data2021['Founded'].astype(int)
data2021['Founded'].head()

0    2019
1    2015
2    2012
3    2015
4    2021
Name: Founded, dtype: int32

In [447]:

# data2021['Founded'] = data2021['Founded'].apply(str).apply(lambda x: x.split('.')[0])
# data2021['Founded'] = pd.to_numeric(data2021['Founded'])
# data2021.head()

In [448]:
# Making sure the conversion was done

data2021['Founded'].dtype

dtype('int32')

In [449]:
data2021['Founded'].unique()

array([2019, 2015, 2012, 2021, 2014, 2018, 2016, 2020, 2010, 2017, 1993,
       2008, 2013, 1999, 1989, 2011, 2009, 2002, 1994, 2006, 2000, 2007,
       1978, 2003, 1998, 1991, 1984, 2004, 2005, 1963])

### Cleaning the 'Amount' col

In [450]:
# Drop additional rows with anomalous values at the Amount col

drop_rows_by_column_values(data2021, 'Amount', ['Upsparks', 'JITO Angel Network, LetsVenture'])

Replacing undisclosed/$undisclosed/$Undisclosed values under the 'Amounts' Column

In [451]:
# Function to replace anomalous amount values with np.nan

def replace_values_with_nan(df, column_name, values_to_replace):
    """
    Replaces specified values in a column of a DataFrame with np.nan.

    Args:
        df (pandas.DataFrame): The DataFrame containing the column to modify.
        column_name (str): The name of the column to replace values in.
        values_to_replace (list or scalar): The value(s) to replace with np.nan. Can be a single value or a list of values.

    Returns:
        pandas.DataFrame: A modified DataFrame with the specified values replaced by np.nan.

    Example:
        # Create a sample DataFrame
        data = {
            'Column1': [1, 2, 3, 4, 5],
            'Column2': ['A', 'B', 'C', 'D', 'E'],
            'Column3': ['X', 'Y', 'Z', 'X', 'Z']
        }

        df = pd.DataFrame(data)

        # Define the column name and values to replace with np.nan
        column_name = 'Column3'
        values_to_replace = ['X', 'Z']

        # Call the replace_values_with_nan function
        df_modified = replace_values_with_nan(df, column_name, values_to_replace)

        # Print the modified DataFrame
        print(df_modified)
    """
    df[column_name] = df[column_name].replace(values_to_replace, np.nan)
    return df


In [452]:
# Invoking the function to replace the missing values with np.nan

data2021 = replace_values_with_nan(data2021, 'Amount', ['$Undisclosed', '$undisclosed', 'undisclosed', 'Undisclosed', 'None'])

# Replacing the undesired characters

data2021['Amount'] = data2021['Amount'].str.replace('[$,]', '')

In [453]:
# Converting the 'Amount' col to float

data2021['Amount'] = pd.to_numeric(data2021['Amount'])

In [454]:
# Checking....

data2021['Amount'].dtype

dtype('float64')

In [455]:
pd.set_option('display.float_format', '{:.1f}'.format)


In [456]:
data2021.describe()

,Founded,Amount
count,1180.0,1040.0
mean,2016.6,172664159.6
std,4.5,4651133125.4
min,1963.0,10000.0
25%,2015.0,1000000.0
50%,2018.0,3700000.0
75%,2020.0,15000000.0
max,2021.0,150000000000.0


Filling missing values in the 'Amount' Col

In [457]:
# Imputing missing values

array = data2021['Amount'].values.reshape(-1,1)
imputer = SimpleImputer(strategy='median')

data2021['Amount'] = imputer.fit_transform(array)

In [458]:
data2021['Amount'] = data2021['Amount'].astype(float)

In [459]:
data2021[data2021['Amount'] == 150000000000.0]

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
73,Alteria Capital,2018,Mumbai,FinTech,Alteria Capital is a Venture debt firm .,Vinod Murali,None,150000000000.0,Debt


In [460]:
top_10_fundings = data2021.groupby('Company_Brand')['Amount'].sum().nlargest(10)

top_10_fundings

Company_Brand
Alteria Capital    150000000000.0
VerSe Innovation     1450000000.0
BYJU'S               1310000000.0
Dream Sports         1240000000.0
Meesho                870000000.0
Zetwerk               870000000.0
OYO                   865000000.0
Swiggy                800000000.0
BharatPe              530200000.0
Ola                   500000000.0
Name: Amount, dtype: float64

In [461]:
data2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000.0,Pre-series A
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000.0,Series D
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000.0,Series C
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000.0,Seed


In [462]:
data2021['Year_Funded'] = 2021

In [463]:
data2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000.0,Pre-series A,2021
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None,2021
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000.0,Series D,2021
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000.0,Series C,2021
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000.0,Seed,2021


### Cleaning 'Stage' Col

Inspecting the unique values in the 'Stage' Column

In [464]:
data2021['Stage'].unique()

array(['Pre-series A', None, 'Series D', 'Series C', 'Seed', 'Series B',
       'Series E', 'Pre-seed', 'Series A', 'Pre-series B', 'Debt',
       'Bridge', 'Seed+', 'Series F2', 'Series A+', 'Series G',
       'Series F', 'Series H', 'Series B3', 'PE', 'Series F1',
       'Pre-series A1', 'Early seed', 'Series D1', 'Seies A',
       'Pre-series', 'Series A2', 'Series I'], dtype=object)

Check how many records have the null value at the 'Stage' column

In [465]:
data2021['Stage'].isnull().sum()

416

Oops! Pretty much! Let's take a deeper look at those records.

In [466]:
data2021[data2021['Stage'].isnull()].head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None,2021
5,Urban Company,2014,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,188000000.0,None,2021
6,Comofi Medtech,2018,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI",200000.0,None,2021
8,Vitra.ai,2020,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,3700000.0,None,2021
9,Taikee,2010,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,1000000.0,None,2021


Alright! These companies are listed on cruchbase, the leading provider of private-company prospecting and research solutions. <br>
Let's see if we can get reliable data to fill-in the nulls

In [467]:
data2021.iloc[1, 8] = 'Venture - Series Unknown'
data2021.iloc[5, 8] = 'Secondary Market'
data2021.iloc[6, 8] = 'Pre-Seed'
data2021.iloc[8, 8] = 'Seed'
data2021.iloc[9, 8] = 'Venture - Series Unknown'
data2021.iloc[11, 8] = 'Pre-Seed'
data2021.iloc[13, 8] = 'Series A'
data2021.iloc[14, 8] = 'Series D'
data2021.iloc[15, 8] = 'Series E'
data2021.iloc[16, 8] = 'Series A'
data2021.iloc[24, 8] = 'Debt'
data2021.iloc[31, 8] = ' Series A'
data2021.iloc[34, 8] = 'Venture - Series Unknown'
data2021.iloc[35, 8] = 'Venture - Series Unknown'
data2021.iloc[36, 8] = 'Venture - Series Unknown'
data2021.iloc[37, 8] = 'Venture - Series Unknown'
data2021.iloc[40, 8] = 'Series A'
data2021.iloc[42, 8] = 'Seed'
data2021.iloc[46, 8] = 'Equity Crowdfunding'

Good! I could just get a few. 

Let's have a look at what the unique values are again.

In [468]:
data2021['Stage'].unique()

array(['Pre-series A', 'Venture - Series Unknown', 'Series D', 'Series C',
       'Seed', 'Secondary Market', 'Pre-Seed', 'Series A', 'Series E',
       'Series B', 'Pre-seed', 'Debt', ' Series A', None,
       'Equity Crowdfunding', 'Pre-series B', 'Bridge', 'Seed+',
       'Series F2', 'Series A+', 'Series G', 'Series F', 'Series H',
       'Series B3', 'PE', 'Series F1', 'Pre-series A1', 'Early seed',
       'Series D1', 'Seies A', 'Pre-series', 'Series A2', 'Series I'],
      dtype=object)

Ok, Let's put these values in a standard format.

In [469]:

def update_value(value):
    replacements = {
        r'Pre series|Early seed|Pre-series A|Pre-series A1': 'Pre-series',
        r'Seies A|Seed+|Pre-series B|Series A2': 'Series A',
        r'PE': 'Private Equity',
        r'Debt': 'Debt Financing',
        r'Seed1': 'Seed',
        r'None': 'Venture - Series Unknown',
        r'Series A+|Series B3| Series B': 'Series B',
        r'Series F2|Series F1|Series D1|Series D|Series G|Series H|Series I|Series E|Series F': 'Series C',
    }
    
    for pattern, replacement in replacements.items():
        value = re.sub(pattern, replacement, str(value))
    
    return value

data2021 = data2021.applymap(update_value)


Great! Checking........

In [470]:
data2021['Stage'].unique()

array(['Pre-series', 'Venture - Series Unknown', 'Series C', 'Series B',
       'Secondary Market', 'Pre-Series B', 'Pre-seed', 'Debt Financing',
       ' Series B', 'Equity Crowdfunding', 'Bridge', 'Series B+',
       'Private Equity', 'Pre-series1'], dtype=object)

Ouch!  'Series B+',  'Pre-Series B' and 'Pre-series1' want to be treated diferently! I see. Let's implore another method!

In [471]:
data2021['Stage'].replace('Series B+', 'Series B', inplace=True)
data2021['Stage'].replace('Pre-series1', 'Pre-series', inplace=True)
data2021['Stage'].replace('Pre-Series B', 'Series A', inplace=True)
data2021['Stage'].replace(' Series B', 'Series B', inplace=True)


Nice! Let's take a look at the unique values for stage column again!

In [472]:
data2021['Stage'].unique()

array(['Pre-series', 'Venture - Series Unknown', 'Series C', 'Series B',
       'Secondary Market', 'Series A', 'Pre-seed', 'Debt Financing',
       'Equity Crowdfunding', 'Bridge', 'Private Equity'], dtype=object)

Perfect!

Dealing with the Investor Column <br>

Inspecting the unique instances of the 'Investor Column'

In [473]:
data2021['Investor'].unique()

array(['BEENEXT, Entrepreneur First',
       'Unilazer Ventures, IIFL Asset Management',
       'GSV Ventures, Westbridge Capital', 'CDC Group, IDG Capital',
       'Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal', 'Vy Capital',
       'CIIE.CO, KIIT-TBI', 'Inflection Point Ventures',
       'Inflexor Ventures', 'Venture - Series Unknown',
       '9Unicorns Accelerator Fund, Metaform Ventures',
       'SucSEED Indovation, IIM Calcutta Innovation Park',
       'Safe Planet Medicare', 'Impact Partners, C4D Partners',
       'Tiger Global Management, InnoVen Capital', 'Novo Tellus Capital',
       'Raintree Family Office, ADB arm',
       'Mumbai Angels, Narendra Shyamsukha', 'Paradigm, Kunal Shah',
       'Matrix Partners India, GIC', 'Chiratae Ventures, JAFCO Asia',
       'Mumbai Angels Network, Expert DOJO', 'GVFL',
       'Kotak Mahindra Bank, FMO', 'Kalaari Capital',
       'NB Ventures, IAN Fund',
       'Sequoia Capital India, Hummingbird Ventures',
       'Gaurav Munjal, Snehil Kh

NO anomaly spotted here, good! Let's now separate and keep first names......nooooo....wait! <br>

..before that I just saw 'http://100x.vc/' as an investor. Let's check that out!

In [474]:
# Get the row/rows that have 'http://100x.vc/' as investor

data2021[data2021['Investor'] == 'http://100x.vc/']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded
59,Karnival,2015,Bangalore,SaaS startup,A SAAS platform for retailers and brands to cr...,Neeraj Tyagi,http://100x.vc/,2500000.0,Series B,2021


Alright, let's dig deeper by clicking the link.......Ohh, I got it! The Investor is a company named '100x.vc' <br>

Making the change........

In [482]:
# Slice that value and asign '100x.vc'

data2021.at[59, 'Investor'] = '100x.vc'

In [483]:
# Checking..........
data2021[data2021['Investor'] == '100x.vc']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,Year_Funded,6
59,Karnival,2015,Bangalore,SaaS startup,A SAAS platform for retailers and brands to cr...,Neeraj Tyagi,100x.vc,2500000.0,Series B,2021,100x.vc


Good! What next?............ Let's now clean by retaining only the first

In [477]:
# Splitting the names separated by ',' and keeping the first

data2021["Investor"] = data2021["Investor"].map(lambda x: str(x).split(',')[0])


In [478]:
data2021['Investor'].unique().tolist()

['BEENEXT',
 'Unilazer Ventures',
 'GSV Ventures',
 'CDC Group',
 'Liberatha Kallat',
 'Vy Capital',
 'CIIE.CO',
 'Inflection Point Ventures',
 'Inflexor Ventures',
 'Venture - Series Unknown',
 '9Unicorns Accelerator Fund',
 'SucSEED Indovation',
 'Safe Planet Medicare',
 'Impact Partners',
 'Tiger Global Management',
 'Novo Tellus Capital',
 'Raintree Family Office',
 'Mumbai Angels',
 'Paradigm',
 'Matrix Partners India',
 'Chiratae Ventures',
 'Mumbai Angels Network',
 'GVFL',
 'Kotak Mahindra Bank',
 'Kalaari Capital',
 'NB Ventures',
 'Sequoia Capital India',
 'Gaurav Munjal',
 'JITO Angel Network',
 'STRIVE VC',
 'Venture Catalysts',
 'Jaipuria Family Office',
 'Amazon',
 'Riso Capital',
 'MTG Ventures',
 'Qualcomm Ventures',
 '9Unicorns',
 'Morgan Stanley',
 'Alfa Ventures',
 'Stellaris Venture Partners',
 'Spanache',
 'DSG Consumer Partners',
 'Hindustan Media Ventures',
 'Angelbay Holdings',
 'Alteria Capital',
 'WaterBridge Ventures',
 'Carmignac',
 'Vaibhav Domkundwar',
 'A

Done!.............let's check further for missing values

In [479]:
data2021['Investor'].isnull().sum()

0

In [480]:
data2021['Sector'].unique()

array(['AI startup', 'EdTech', 'B2B E-commerce', 'FinTech',
       'Home services', 'HealthTech', 'Tech Startup', 'E-commerce',
       'B2B service', 'Helathcare', 'Renewable Energy', 'Electronics',
       'IT startup', 'Food & Beverages', 'Aeorspace', 'Deep Tech',
       'Dating', 'Gaming', 'Robotics', 'Retail', 'Food', 'Oil and Energy',
       'AgriTech', 'Telecommuncation', 'Milk startup', 'AI Chatbot', 'IT',
       'Logistics', 'Hospitality', 'Fashion', 'Marketing',
       'Transportation', 'LegalTech', 'Food delivery', 'Automotive',
       'SaaS startup', 'Fantasy sports', 'Video communication',
       'Social Media', 'Skill development', 'Rental', 'Recruitment',
       'HealthCare', 'Sports', 'Computer Games', 'Consumer Goods',
       'Information Technology', 'Apparel & Fashion',
       'Logistics & Supply Chain', 'Healthtech', 'Healthcare',
       'SportsTech', 'HRTech', 'Wine & Spirits',
       'Mechanical & Industrial Engineering', 'Spiritual',
       'Financial Services', 'I

In [481]:
data2019


,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding
3,HomeLane,2014.0,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath","Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D
4,Nu Genes,2004.0,Telangana,AgriTech,"It is a seed company engaged in production, pr...",Narayana Reddy Punyala,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN
5,FlytBase,NaN,Pune,Technology,A drone automation platform,Nitin Gupta,Undisclosed,Undisclosed,NaN
6,Finly,NaN,Bangalore,SaaS,It builds software products that makes work si...,"Vivek AG, Veekshith C Rai","Social Capital, AngelList India, Gemba Capital...",Undisclosed,NaN
7,Kratikal,2013.0,Noida,Technology,It is a product-based cybersecurity solutions ...,"Pavan Kushwaha, Paratosh Bansal, Dip Jung Thapa","Gilda VC, Art Venture, Rajeev Chitrabhanu.","$1,000,000",Pre series A
8,Quantiphi,NaN,NaN,AI & Tech,It is an AI and big data services company prov...,Renuka Ramnath,Multiples Alternate Asset Management,"$20,000,000",Series A
9,Lenskart,2010.0,Delhi,E-commerce,It is a eyewear company,"Peyush Bansal, Amit Chaudhary, Sumeet Kapahi",SoftBank,"$275,000,000",Series G
